In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.head(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,7.5,2015
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,7.3,2015


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.info()  # Проверяем, есть ли пропуски в данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

data['profit'] = data['revenue'] - data['budget'] # Добавление колонки с прибылью
data['len_title'] = data['original_title'].apply(lambda x: len(x))  # Добавление колонки с длиной названия фильма

# разделение данных в колонках с несколькими строковыми значениями для дальннейшей работы
data['genres'] = data['genres'].str.split('|')
data['cast'] = data['cast'].str.split('|')
data['director'] = data['director'].str.split('|')
data['production_companies'] = data['production_companies'].str.split('|')
data['words_in_overview'] = data['overview'].apply(lambda x: len(x.split(' ')))

def month(x):
    """Функция для преобразования номера месяца (из типа str) в название месяца"""
    if x == '1':
        return 'January'
    elif x == '2':
        return 'February'
    elif x == '3':
        return 'March'
    elif x == '4':
        return 'April'
    elif x == '5':
        return 'May'
    elif x == '6':
        return 'June'
    elif x == '7':
        return 'July'
    elif x == '8':
        return 'August'
    elif x == '9':
        return 'September'
    elif x == '10':
        return 'October'
    elif x == '11':
        return 'November'
    else:
        return 'December'

    
data['month'] = data['release_date'].str.split('/').apply(lambda x: month(x[0]))  # Добавление колонки с названием месяца 

# 1. У какого фильма из списка самый большой бюджет?

In [6]:
data[data['budget'] == data['budget'].max()]['original_title']

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

In [7]:
answers['1'] = data[data['budget'] == data['budget'].max()]['original_title'].iloc[0]  # "+"

# 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
data[data['runtime'] == data['runtime'].max()]['original_title']

1157    Gods and Generals
Name: original_title, dtype: object

In [9]:
answers['2'] = data[data['runtime'] == data['runtime'].max()]['original_title'].iloc[0]  # "+"

# 3. Какой из фильмов самый короткий (в минутах)?





In [10]:
data[data['runtime'] == data['runtime'].min()]['original_title']

768    Winnie the Pooh
Name: original_title, dtype: object

In [11]:
answers['3'] = data[data['runtime'] == data['runtime'].min()]['original_title'].iloc[0]  # "+"

# 4. Какова средняя длительность фильмов?


In [12]:
data['runtime'].mean()

109.6585494970884

In [13]:
answers['4'] = data['runtime'].mean()  # "+"

ВАРИАНТ 2

In [14]:
data.describe().runtime['mean']

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [15]:
data.describe().runtime['50%']
data['runtime'].median()

107.0

In [16]:
answers['5'] = data['runtime'].median()  # "+"

ВАРИАНТ 2

In [17]:
data.describe().runtime['50%']

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [18]:
data[data['profit'] == data['profit'].max()]['original_title']

239    Avatar
Name: original_title, dtype: object

In [19]:
answers['6'] = data[data['profit'] == data['profit'].max()]['original_title'].iloc[0]  # "+"

# 7. Какой фильм самый убыточный? 

In [20]:
data[data['profit'] == data['profit'].min()]['original_title']

1245    The Lone Ranger
Name: original_title, dtype: object

In [21]:
answers['7'] = data[data['profit'] == data['profit'].min()]['original_title'].iloc[0]  # "+"

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [22]:
len(data[data['profit'] > 0])

1478

In [23]:
answers['8'] = len(data[data['profit'] > 0])  # "+"

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [24]:
data_2008 = data[data['release_year'] == 2008]
data_2008[data_2008['profit'] == data_2008['profit'].max()]['original_title']

599    The Dark Knight
Name: original_title, dtype: object

In [25]:
answers['9'] = data_2008[data_2008['profit'] == data_2008['profit'].max()]['original_title'].iloc[0]  # "+"

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [26]:
data_201222014 = data[((data['release_year'] >= 2012) & (data['release_year'] <= 2014))]
data_201222014[data_201222014['profit'] == data_201222014['profit'].min()]['original_title']

1245    The Lone Ranger
Name: original_title, dtype: object

In [27]:
answers['10'] = data_201222014[data_201222014['profit'] == data_201222014['profit'].min()]['original_title'].iloc[0]  # "+"

# 11. Какого жанра фильмов больше всего?

In [28]:
data_genres = data.explode('genres')
data_genres['genres'].value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
Name: genres, dtype: int64

In [29]:
answers['11'] = data_genres['genres'].value_counts().index[0]  # "+"

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [30]:
data_genres[data_genres['profit'] > 0]['genres'].value_counts()

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
Name: genres, dtype: int64

In [31]:
answers['12'] = data_genres[data_genres['profit'] > 0]['genres'].value_counts().index[0]  # "+"

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [32]:
data_director = data.explode('director') 
data_director.groupby(['director']).revenue.sum().sort_values(ascending=False)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
                        ...    
David MichÃ´d           2295423
Steven Shainberg        2281089
Paul Schrader           2062066
Keanu Reeves            2054941
Simon Hunter            2033165
Name: revenue, Length: 997, dtype: int64

In [33]:
answers['13'] = data_director.groupby(['director']).revenue.sum().idxmax()  # "+"

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [34]:
data_action = data_genres[data_genres['genres'] == 'Action']
data_act_dir = data_action.explode('director')
data_act_dir['director'].value_counts()

Robert Rodriguez       9
Paul W.S. Anderson     7
Michael Bay            7
Ridley Scott           6
Antoine Fuqua          6
                      ..
Randall Wallace        1
Peter Segal            1
Motonori Sakakibara    1
Brian Robbins          1
Gareth Edwards         1
Name: director, Length: 364, dtype: int64

In [35]:
answers['14'] = data_act_dir['director'].value_counts().index[0]  # "+"

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [36]:
data_2012 = data[data.release_year == 2012].copy()
data_cast2012 = data_2012.explode('cast')
data_cast2012.groupby(['cast']).revenue.sum().sort_values(ascending=False)

cast
Chris Hemsworth      2027450773
Denis Leary          1629460639
Anne Hathaway        1522851057
Chris Evans          1519557910
Robert Downey Jr.    1519557910
                        ...    
Jason Bateman           3428048
Danny Huston            2106557
Sami Gayle              2106557
Josh Lucas              2106557
Nicolas Cage            2106557
Name: revenue, Length: 466, dtype: int64

In [37]:
answers['15'] = data_cast2012.groupby(['cast']).revenue.sum().idxmax()  # "+"

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [38]:
data_mome = data[data['budget'] > data['budget'].mean()]  ## "mome" - more mean
data_cast_mome = data_mome.explode('cast')
data_cast_mome['cast'].value_counts()

Matt Damon           18
Adam Sandler         17
Angelina Jolie       16
Eddie Murphy         15
Samuel L. Jackson    15
                     ..
Mary Gibbs            1
Jay Hernandez         1
Angourie Rice         1
Peter Berg            1
Stephen Baldwin       1
Name: cast, Length: 1505, dtype: int64

In [39]:
answers['16'] = data_cast_mome['cast'].value_counts().index[0]  # "+"

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [40]:
data_cast = data.explode('cast')
data_Cage = data_cast[data_cast['cast'] == 'Nicolas Cage']
data_genres_Cage = data_Cage.explode('genres')
data_genres_Cage['genres'].value_counts()

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Family              3
Animation           3
Mystery             3
History             2
Romance             1
War                 1
Horror              1
Name: genres, dtype: int64

In [41]:
answers['17'] = data_genres_Cage['genres'].value_counts().index[0]  # "+"

# 18. Самый убыточный фильм от Paramount Pictures

In [42]:
data_companies = data.explode('production_companies')
data_Paramount = data_companies[data_companies['production_companies'] == 'Paramount Pictures']
data_Paramount[['original_title', 'profit']].sort_values(['profit'])['original_title']

925                                  K-19: The Widowmaker
1123                                             Timeline
1722                                                 Next
1623                                                Alfie
1631                                              Twisted
                              ...                        
730                  Mission: Impossible - Ghost Protocol
1738                                         Transformers
603     Indiana Jones and the Kingdom of the Crystal S...
317                   Transformers: Revenge of the Fallen
828                        Transformers: Dark of the Moon
Name: original_title, Length: 122, dtype: object

In [43]:
answers['18'] = data_Paramount[['original_title', 'profit']].sort_values(['profit'])['original_title'].iloc[0]  # "+"

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [44]:
data.groupby('release_year')['revenue'].sum().sort_values(ascending=False)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

In [45]:
answers['19'] = data.groupby('release_year')['revenue'].sum().idxmax()  # "+"

# 20. Какой самый прибыльный год для студии Warner Bros?

In [46]:
data_Warner = data_companies[data_companies['production_companies'].str.contains ("Warner")]
data_Warner.groupby('release_year')['profit'].sum().sort_values(ascending=False)

release_year
2014    2292949646
2007    2208217007
2008    2134595031
2010    1974712985
2011    1871393682
2003    1844008221
2009    1822454136
2013    1636453400
2004    1631933725
2005    1595278228
2001    1282142579
2012    1258020056
2002    1022709901
2015     870368348
2006     622332868
2000     452631386
Name: profit, dtype: int64

In [47]:
answers['20'] = data_Warner.groupby('release_year')['profit'].sum().idxmax()  # "+"

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [48]:
data['month'].value_counts().sort_values(ascending=False)

September    227
December     190
October      186
August       161
March        156
April        149
June         147
November     146
July         142
May          140
February     135
January      110
Name: month, dtype: int64

In [49]:
answers['21'] = data['month'].value_counts().idxmax()  # "+"

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [50]:
data[((data['month'] == 'June')|
      (data['month'] == 'July')|
      (data['month'] == 'August'))]['imdb_id'].nunique()

450

In [51]:
answers['22'] = data[((data['month'] == 'June')|
                      (data['month'] == 'July')|
                      (data['month'] == 'August'))]['imdb_id'].nunique()  # "+"

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [52]:
data_director[((data['month'] == 'December')|
               (data['month'] == 'January')|
               (data['month'] == 'February'))]['director'].value_counts()

<ipython-input-52-aa56e07c158e>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_director[((data['month'] == 'December')|


Peter Jackson                     7
Steven Soderbergh                 6
Clint Eastwood                    6
Martin Scorsese                   4
Nancy Meyers                      4
                                 ..
Alejandro GonzÃ¡lez IÃ±Ã¡rritu    1
Uwe Boll                          1
Albert Hughes                     1
Thor Freudenthal                  1
David Dobkin                      1
Name: director, Length: 358, dtype: int64

In [53]:
answers['23'] = data_director[((data['month'] == 'December')|
                               (data['month'] == 'January')|
                               (data['month'] == 'February'))]['director'].value_counts().index[0]

<ipython-input-53-c0d1020768dc>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  answers['23'] = data_director[((data['month'] == 'December')|


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [54]:
data_companies.groupby(['production_companies'])['len_title'].mean().sort_values(ascending=False)

production_companies
Four By Two Productions       83.0
Jim Henson Company, The       59.0
Dos Corazones                 47.0
Museum Canada Productions     46.0
Polsky Films                  46.0
                              ... 
Everest Entertainment          3.0
Berlanti Productions           3.0
XM2 Productions                2.0
Ixtlan Productions             2.0
Global Entertainment Group     2.0
Name: len_title, Length: 1771, dtype: float64

In [55]:
answers['24'] = data_companies.groupby(['production_companies'])['len_title'].mean().idxmax()

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [56]:
data_companies.groupby(['production_companies'])['words_in_overview'].mean().sort_values(ascending=False)

production_companies
Midnight Picture Show                    175.0
Room 9 Entertainment                     161.0
Heineken Branded Entertainment           161.0
98 MPH Productions                       159.0
Brookwell-McNamara Entertainment         156.0
                                         ...  
London Boulevard                          13.0
Phantom Four                              13.0
Henceforth                                13.0
Empire Pictures                           11.0
Motion Picture Corporation of America     11.0
Name: words_in_overview, Length: 1771, dtype: float64

In [57]:
answers['25'] = data_companies.groupby(['production_companies'])['words_in_overview'].mean().idxmax()

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 


In [58]:
data[data['vote_average'] > data['vote_average'].quantile(0.99)]['original_title']

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

In [59]:
answers['26'] = data[data['vote_average'] > data['vote_average'].quantile(0.99)]['original_title']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [60]:
data['pairs'] = data['cast'].apply(lambda x: list(combinations(x,2)))
data_pairs = data.explode('pairs')
Counter(data_pairs['pairs']).most_common(1)

[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

In [61]:
answers['27'] = Counter(data_pairs['pairs']).most_common(1)[0][0]

# Submission

In [62]:
answers  # в конце можно посмотреть свои ответы к каждому вопросу

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': 109.6585494970884,
 '5': 107.0,
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': 1478,
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20': 2014,
 '21': 'September',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 9                                          Inside Out
 34                                               Room
 118                                      Interstellar
 119                           Guardians of the Galaxy
 125                                The Imitation Game
 128                                         Gone Girl
 138                          The Grand Budapest Hotel
 370                                

In [63]:
len(answers)  # и убедиться что ни чего не пропустил)

27